# Customer Analysis BiPi

## Instrucciones/preguntas:

Dado el dataset que se adjunta, contestar las siguientes preguntas:

- ¿Qué método de pandas utilizarías para cargar la información en un dataframe?

- ¿Qué alternativa a pandas podrías utilizar?

- ¿Qué procedimientos harías para hacer una limpieza de los datos?

Escribe un script en Python apoyado en pandas y en cualquier librería de graficación que conozcas que en tiempo de ejecución realice lo siguiente:

Nota: El script tiene que funcionar sin tener el dataset en local. Es decir, debes subir el dataset a algún sitio (git, drive...) y ponerlo en modo público.


1. Calcular el número y porcentaje sobre el total absoluto de Mujeres monoparentales que han gastado más de 15.000 dólares.

2. Calcular cuántos abogados (M/F) hay en el dataset que tengan mayor experiencia laboral que la media de ingenieros hombres.

3. Dibuja la distribución de abogados en función de su gasto anual (utiliza franjas de gasto) y de su sexo.

4. Añade algún análisis que creas que puede ser interesante explicando por pantalla el resultado del mismo